# **10. Veri Tabanı İşlemleri (SQLite)**

# **10.1 SQLite Nedir?**

**SQLite**, dosya tabanlı, hafif ve sunucu gerektirmeyen bir veri tabanı sistemidir.

## Neden SQLite?

| Özellik | Açıklama |
|---------|----------|
| **Hafif** | Kurulum gerektirmez |
| **Taşınabilir** | Tek dosyada saklanır (.db) |
| **Entegre** | Python'da `sqlite3` modülü ile hazır |
| **SQL Uyumlu** | Standart SQL sorguları |

## Temel Kavramlar

| Kavram | Açıklama |
|--------|----------|
| **Veri Tabanı (Database)** | Verilerin saklandığı dosya |
| **Tablo (Table)** | Verilerin organize edildiği yapı |
| **Satır (Row)** | Bir kayıt |
| **Sütun (Column)** | Bir alan/nitelik |
| **SQL** | Sorgulama dili |

---

---

---

# **10.2 Veri Tabanı Bağlantısı**

`sqlite3.connect()` ile veri tabanına bağlanılır. Dosya yoksa otomatik oluşturulur.

```python
import sqlite3
vt = sqlite3.connect('veritabani.db')  # Bağlantı
im = vt.cursor()                        # İmleç (cursor)
# İşlemler...
vt.commit()                             # Değişiklikleri kaydet
vt.close()                              # Bağlantıyı kapat
```

In [ ]:
# sqlite3 modülünü import et
# Python'ın standart kütüphanesinde hazır gelir

import sqlite3

# Veri tabanı bağlantısı oluştur
# Dosya yoksa otomatik oluşturulur
vt = sqlite3.connect('ornek.db')  # ornek.db dosyası

# İmleç (cursor) oluştur
# SQL komutlarını çalıştırmak için kullanılır
im = vt.cursor()

print("Veri tabanına bağlandı!")
print(f"Bağlantı tipi: {type(vt)}")
print(f"İmleç tipi: {type(im)}")

---

---

---

# **10.3 Tablo Oluşturma (CREATE TABLE)**

`CREATE TABLE` komutu ile tablo oluşturulur.

## Veri Tipleri

| SQLite Tipi | Python Karşılığı |
|-------------|------------------|
| `INTEGER` | `int` |
| `REAL` | `float` |
| `TEXT` | `str` |
| `BLOB` | `bytes` |
| `NULL` | `None` |

In [ ]:
# Tablo oluşturma
# IF NOT EXISTS: Tablo varsa hata vermez

# SQL komutu - çok satırlı string
sql_tablo = """
CREATE TABLE IF NOT EXISTS personel (
    no INTEGER PRIMARY KEY,
    isim TEXT NOT NULL,
    statu TEXT,
    yas INTEGER,
    maas REAL
)
"""

# execute() ile SQL komutunu çalıştır
im.execute(sql_tablo)

# commit() ile değişiklikleri kalıcı yap
vt.commit()

print("'personel' tablosu oluşturuldu!")

In [ ]:
# Tablo yapısını görüntüleme
# pragma table_info ile sütun bilgileri alınır

im.execute("PRAGMA table_info(personel)")
sutunlar = im.fetchall()

print("=== personel Tablo Yapısı ===")
print(f"{'Sütun':<10} {'Tip':<10} {'Null?':<8} {'PK':<5}")
print("-" * 35)
for sutun in sutunlar:
    # sutun: (id, name, type, notnull, default, pk)
    print(f"{sutun[1]:<10} {sutun[2]:<10} {'No' if sutun[3] else 'Yes':<8} {'Evet' if sutun[5] else '-':<5}")

---

---

---

# **10.4 Veri Ekleme (INSERT)**

`INSERT INTO` komutu ile veri eklenir.

## Güvenli Parametre Kullanımı

SQL Injection saldırılarına karşı **parametreli sorgular** kullanın:

```python
# GEÇERSİZ: String birleştirme (tehlikeli!)
im.execute(f"INSERT INTO tablo VALUES ({deger})")

# GEÇERLİ: Parametreli (güvenli)
im.execute("INSERT INTO tablo VALUES (?)", (deger,))
```

In [ ]:
# Tek kayıt ekleme
# Parametreli sorgu: ? ile yer tutucu kullan

# Veriler
no = 1
isim = "Züber"
statu = "Müdür"
yas = 46
maas = 50000.0

# Parametreli INSERT
sql = "INSERT INTO personel (no, isim, statu, yas, maas) VALUES (?, ?, ?, ?, ?)"
im.execute(sql, (no, isim, statu, yas, maas))  # Tuple olarak geç

vt.commit()
print(f"'{isim}' eklendi!")

In [ ]:
# Çoklu kayıt ekleme - executemany()
# Liste içinde tuple'lar

veriler = [
    (2, "Uygar", "Mühendis", 21, 35000.0),
    (3, "İdil", "Doktor", 25, 45000.0),
    (4, "Ali", "Teknisyen", 30, 25000.0),
    (5, "Ayşe", "Analist", 28, 38000.0)
]

sql = "INSERT INTO personel (no, isim, statu, yas, maas) VALUES (?, ?, ?, ?, ?)"

# executemany() ile toplu ekleme
im.executemany(sql, veriler)

vt.commit()
print(f"{len(veriler)} kayıt eklendi!")

---

---

---

# **10.5 Veri Okuma (SELECT)**

`SELECT` komutu ile veri sorgulanır.

## Veri Alma Metotları

| Metot | Açıklama |
|-------|----------|
| `fetchone()` | Tek satır döndür |
| `fetchall()` | Tüm satırları döndür |
| `fetchmany(n)` | n satır döndür |

In [ ]:
# Tüm verileri seç - SELECT *
# * tüm sütunları seçer

im.execute("SELECT * FROM personel")

# fetchall() tüm sonuçları liste olarak döndürür
veriler = im.fetchall()

print("=== Tüm Personel ===")
print(f"{'No':<5} {'İsim':<10} {'Statü':<12} {'Yaş':<5} {'Maaş':<10}")
print("-" * 45)

for veri in veriler:
    # veri: (no, isim, statu, yas, maas)
    print(f"{veri[0]:<5} {veri[1]:<10} {veri[2]:<12} {veri[3]:<5} {veri[4]:<10.2f}")

In [ ]:
# Koşullu seçim - WHERE
# Belirli kriterlere göre filtrele

# 30 yaş altı personel
im.execute("SELECT isim, yas, statu FROM personel WHERE yas < 30")
gencler = im.fetchall()

print("=== 30 Yaş Altı Personel ===")
for p in gencler:
    print(f"  {p[0]} - {p[1]} yaşında - {p[2]}")

print()

# Maaşı 40000 üstü personel
im.execute("SELECT isim, maas FROM personel WHERE maas > 40000")
yuksek_maas = im.fetchall()

print("=== Yüksek Maaşlı Personel ===")
for p in yuksek_maas:
    print(f"  {p[0]} - {p[1]:,.2f} TL")

In [ ]:
# Sıralama ve limit - ORDER BY, LIMIT
# Sonuçları sırala ve sınırla

# Maaşa göre azalan sırada ilk 3
im.execute("""
    SELECT isim, statu, maas 
    FROM personel 
    ORDER BY maas DESC 
    LIMIT 3
""")

en_yuksek = im.fetchall()

print("=== En Yüksek Maaşlı 3 Kişi ===")
for i, p in enumerate(en_yuksek, 1):
    print(f"  {i}. {p[0]} ({p[1]}) - {p[2]:,.2f} TL")

In [ ]:
# Gruplama ve fonksiyonlar - GROUP BY, COUNT, AVG, SUM
# İstatistiksel sorgular

# Toplam personel sayısı
im.execute("SELECT COUNT(*) FROM personel")
toplam = im.fetchone()[0]
print(f"Toplam personel: {toplam}")

# Ortalama maaş
im.execute("SELECT AVG(maas) FROM personel")
ortalama = im.fetchone()[0]
print(f"Ortalama maaş: {ortalama:,.2f} TL")

# Toplam maaş
im.execute("SELECT SUM(maas) FROM personel")
toplam_maas = im.fetchone()[0]
print(f"Toplam maaş: {toplam_maas:,.2f} TL")

# En düşük ve en yüksek yaş
im.execute("SELECT MIN(yas), MAX(yas) FROM personel")
yas_araligi = im.fetchone()
print(f"Yaş aralığı: {yas_araligi[0]} - {yas_araligi[1]}")

---

---

---

# **10.6 Veri Güncelleme (UPDATE)**

`UPDATE` komutu ile mevcut veriler değiştirilir.

```sql
UPDATE tablo SET sutun = deger WHERE kosul
```

**DİKKAT**: `WHERE` olmadan tüm satırlar güncellenir!

In [ ]:
# Tek kayıt güncelleme
# WHERE koşulu önemli!

# Züber'in maaşını güncelle
yeni_maas = 55000.0
personel_no = 1

im.execute("""
    UPDATE personel 
    SET maas = ? 
    WHERE no = ?
""", (yeni_maas, personel_no))

vt.commit()

# rowcount: etkilenen satır sayısı
print(f"{im.rowcount} kayıt güncellendi!")

# Kontrolü: Güncel veriyi göster
im.execute("SELECT * FROM personel WHERE no = ?", (personel_no,))
print(f"Güncel kayıt: {im.fetchone()}")

In [ ]:
# Birden fazla alan güncelleme
# SET sutun1 = deger1, sutun2 = deger2

# Uygar'ın statüsünü ve maaşını güncelle
im.execute("""
    UPDATE personel 
    SET statu = ?, maas = ? 
    WHERE isim = ?
""", ("Kıdemli Mühendis", 42000.0, "Uygar"))

vt.commit()
print(f"{im.rowcount} kayıt güncellendi!")

# Kontrolü
im.execute("SELECT * FROM personel WHERE isim = 'Uygar'")
print(f"Güncel kayıt: {im.fetchone()}")

---

---

---

# **10.7 Veri Silme (DELETE)**

`DELETE FROM` komutu ile kayıt silinir.

```sql
DELETE FROM tablo WHERE kosul
```

**DİKKAT**: `WHERE` olmadan tüm veriler silinir!

In [ ]:
# Tek kayıt silme
# WHERE koşulu zorunlu!

# Ali'yi sil
im.execute("DELETE FROM personel WHERE isim = ?", ("Ali",))

vt.commit()
print(f"{im.rowcount} kayıt silindi!")

# Kalan kayıtları göster
im.execute("SELECT no, isim FROM personel")
print(f"\nKalan personel: {im.fetchall()}")

---

---

---

# **10.8 with İfadesi ile Güvenli Kullanım**

`with` ifadesi bağlantıyı **otomatik kapatır** ve **hata durumunda rollback** yapar.

```python
with sqlite3.connect('vt.db') as baglanti:
    im = baglanti.cursor()
    # İşlemler...
# Otomatik commit ve close
```

In [ ]:
# with ifadesi ile güvenli kullanım
# Bağlantı otomatik kapanır, hata durumunda rollback olur

with sqlite3.connect('ornek.db') as vt:
    im = vt.cursor()
    
    # Tüm personeli listele
    im.execute("SELECT * FROM personel")
    veriler = im.fetchall()
    
    print("=== Personel Listesi (with ile) ===")
    for veri in veriler:
        print(f"  {veri}")

# Blok dışına çıkınca bağlantı otomatik kapandı!
print("\nBağlantı otomatik kapandı.")

---

---

---

# **10.9 Pratik Örnekler**

In [ ]:
# Pratik Örnek 1: Ürün Yönetim Sistemi
# CRUD işlemleri içeren basit bir sistem

class UrunYonetimi:
    """Ürün yönetim sistemi."""
    
    def __init__(self, db_dosya):
        """Veri tabanı bağlantısı ve tablo oluştur."""
        self.db_dosya = db_dosya
        self._tablo_olustur()
    
    def _tablo_olustur(self):
        """Ürünler tablosunu oluştur."""
        with sqlite3.connect(self.db_dosya) as vt:
            vt.execute("""
                CREATE TABLE IF NOT EXISTS urunler (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    barkod TEXT UNIQUE NOT NULL,
                    isim TEXT NOT NULL,
                    fiyat REAL NOT NULL,
                    stok INTEGER DEFAULT 0
                )
            """)
    
    def urun_ekle(self, barkod, isim, fiyat, stok=0):
        """Yeni ürün ekle."""
        try:
            with sqlite3.connect(self.db_dosya) as vt:
                vt.execute(
                    "INSERT INTO urunler (barkod, isim, fiyat, stok) VALUES (?, ?, ?, ?)",
                    (barkod, isim, fiyat, stok)
                )
            print(f"'{isim}' eklendi!")
            return True
        except sqlite3.IntegrityError:
            print(f"Hata: Bu barkod zaten kayıtlı!")
            return False
    
    def urunleri_listele(self):
        """Tüm ürünleri listele."""
        with sqlite3.connect(self.db_dosya) as vt:
            im = vt.execute("SELECT * FROM urunler")
            urunler = im.fetchall()
            
        print("\n=== Ürün Listesi ===")
        print(f"{'ID':<4} {'Barkod':<12} {'İsim':<15} {'Fiyat':<10} {'Stok':<5}")
        print("-" * 50)
        for u in urunler:
            print(f"{u[0]:<4} {u[1]:<12} {u[2]:<15} {u[3]:<10.2f} {u[4]:<5}")
    
    def stok_guncelle(self, barkod, miktar):
        """Stok miktarını güncelle."""
        with sqlite3.connect(self.db_dosya) as vt:
            vt.execute(
                "UPDATE urunler SET stok = stok + ? WHERE barkod = ?",
                (miktar, barkod)
            )
        print(f"Stok güncellendi: {miktar:+d} adet")

# Test
uy = UrunYonetimi('magaza.db')

uy.urun_ekle("1234567890", "Kalem", 12.50, 100)
uy.urun_ekle("0987654321", "Defter", 25.00, 50)
uy.urun_ekle("1111111111", "Silgi", 5.00, 200)

uy.urunleri_listele()

print("\nStok güncelleme:")
uy.stok_guncelle("1234567890", -10)  # 10 adet satış
uy.urunleri_listele()

In [ ]:
# Pratik Örnek 2: Not Defteri Uygulaması
# Tarih damgalı notlar

from datetime import datetime

def not_defteri_olustur():
    """Not tablosu oluştur."""
    with sqlite3.connect('notlar.db') as vt:
        vt.execute("""
            CREATE TABLE IF NOT EXISTS notlar (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                baslik TEXT NOT NULL,
                icerik TEXT,
                tarih TEXT DEFAULT CURRENT_TIMESTAMP
            )
        """)

def not_ekle(baslik, icerik):
    """Yeni not ekle."""
    with sqlite3.connect('notlar.db') as vt:
        vt.execute(
            "INSERT INTO notlar (baslik, icerik) VALUES (?, ?)",
            (baslik, icerik)
        )
    print(f"Not eklendi: {baslik}")

def notlari_listele():
    """Tüm notları listele."""
    with sqlite3.connect('notlar.db') as vt:
        im = vt.execute("SELECT * FROM notlar ORDER BY tarih DESC")
        notlar = im.fetchall()
        
    print("\n=== Notlarım ===")
    for n in notlar:
        print(f"\n[{n[0]}] {n[1]}")
        print(f"    {n[2]}")
        print(f"    Tarih: {n[3]}")

# Test
not_defteri_olustur()

not_ekle("Python Öğreniyorum", "Bugün SQLite veritabanı işlemlerini öğrendim.")
not_ekle("Alışveriş Listesi", "Ekmek, süt, yumurta almayı unutma!")
not_ekle("Toplantı Notu", "Pazartesi saat 10:00'da proje toplantısı.")

notlari_listele()

In [ ]:
# Veri tabanı bağlantısını kapat
# (Eğer hala açıksa)

try:
    vt.close()
    print("Veri tabanı bağlantısı kapatıldı.")
except:
    print("Bağlantı zaten kapalı.")

---

## **Özet Tablosu**

### SQL Komutları

| Komut | Açıklama | Örnek |
|-------|----------|-------|
| `CREATE TABLE` | Tablo oluştur | `CREATE TABLE tablo (sutun TIP)` |
| `INSERT INTO` | Veri ekle | `INSERT INTO tablo VALUES (?)` |
| `SELECT` | Veri oku | `SELECT * FROM tablo WHERE kosul` |
| `UPDATE` | Veri güncelle | `UPDATE tablo SET sutun = ? WHERE kosul` |
| `DELETE FROM` | Veri sil | `DELETE FROM tablo WHERE kosul` |

### Python sqlite3 Metotları

| Metot | Açıklama |
|-------|----------|
| `connect()` | Veri tabanına bağlan |
| `cursor()` | İmleç oluştur |
| `execute()` | SQL komutunu çalıştır |
| `executemany()` | Çoklu komut çalıştır |
| `fetchone()` | Tek satır al |
| `fetchall()` | Tüm satırları al |
| `commit()` | Değişiklikleri kaydet |
| `close()` | Bağlantıyı kapat |